In [1]:
import pandas as pd
import json
import csv
import pickle
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [2]:
'''
this function removes the spaces from a given string and returns the new string
'''
def remove_spaces(string):
    return string.replace(" ", "")

# tot_info.json contains the mapping of school address to its latitude and longitude
# this info was gathered offline using Google Maps API

f = open("/Users/23amrutad/Projects/LeadProj/ProjectData/tot_info.json")
lat_long = json.load(f)
lat_long_map = {}
for i in range(len(lat_long)):
    val = lat_long[i]
    key = remove_spaces(val[0].strip()[:-1])
    value = [val[1], val[2]]
    lat_long_map[key] = value


In [3]:
# NH_LEAD.csv contains official lead data published by state of NH, downloaded from <HERE>

'''
given a path, this function reads the csv file at the path
'''
def createLeadDict(path):
    """Read the CSV file and for every school store its name, address, 
        and the lowest lead concentration recorded at the school.
        If the lead level is ND, ignore it.
    """
    
    def make_addr(dataframe, ind):
        a = dataframe["Facility Name"][ind]
        b = dataframe["Town"][ind]
        c = dataframe["State"][ind]
        try:
            address = a + "  " + b + "  " + c
        except Exception as e:
            print('unknown')
            address = "unknown"
        return address
    
    dataframe = pd.read_csv(path)
    print('Number of rows in dataframe = %s'%len(dataframe))
    name_map = {}
    
    for ind in dataframe.index:
        name = dataframe["Facility Name"][ind]
        if name not in name_map.keys():
            addr = make_addr(dataframe, ind)
            name_map[name] = {'addr':addr, 'min_lead_quantity':-1, 'lat':'', 'long':'', 'tract':-1,
                             'water_util_num_viol': -1, 'water_util_pop_size': -1}

            
    for k,v in name_map.items():
        for ind in dataframe.index:
            if k == dataframe["Facility Name"][ind]:
                try:
                    if v['min_lead_quantity'] == -1 or float(dataframe["Result"][ind]) < v['min_lead_quantity']:
                        v['min_lead_quantity'] = float(dataframe["Result"][ind])
                except:
                    pass
    return name_map


name_map = createLeadDict('/Users/23amrutad/Projects/LeadProj/ProjectData/NH_LEAD copy.csv')
for k,v in name_map.items():
    vnospace = remove_spaces(v['addr'])
    if vnospace not in lat_long_map.keys():
        print('un problema %s'%vnospace)
    else:
        lat_long = lat_long_map[vnospace]
        name_map[k]['lat'] = lat_long[0]
        name_map[k]['long'] = lat_long[1]
        #print(lat_long)

Number of rows in dataframe = 23255
un problema ALSTEADPRIMARYSCHOOLALSTEADNH
un problema AUBURNCHILDREN'SHOUSE,AMONTESSORISCHOOLAUBURNNH
un problema CHARLESTOWNPRIMARYSCHOOLCHARLESTOWNNH
un problema LITTLEHANDS,BIGDREAMSCHILDCARECENTERCONWAYNH
un problema EPPINGELEMENTARYSCHOOL,SAUOFFICE,ANDPRESCHOOLEPPINGNH
un problema JUSTLIKEHOMECHILDCAREMANCHESTERNH
un problema GIRLSINCORPORATEDOFNEWHAMPSHIRE,MANCHESTERDIVISIONMANCHESTERNH
un problema NORTHENDMONTESSORISCHOOL,LLCMANCHESTERNH
un problema READY,SET...GROW!CHILDCARE&LEARNINGMARLOWNH
un problema MONADNOCKREGIONALSCHOOLDISTRICT-ACES#93@MT.CAESARSWANZEYNH
un problema HOMEAWAYFROMHOMEWALPOLENH
un problema WALPOLEPRIMARYSCHOOLWALPOLENH


In [4]:
'''
this function takes in the latitude and longitude of a school and returns the tract it's in
'''

def findTract(lat, long, shapes, records):
    for i in range(len(shapes)):
        pts = shapes[i].points
        polygon = Polygon(pts)
        #print(polygon)
        pt = Point(long, lat)
        if polygon.contains(pt):
            res = records[i].TRACTCE
            return res
    return -1

# finding the tract for each school in name_map

# sf is the shapefile downloaded from the census website. it contains information about the boundaries of 
# the tracts from the 2019 5 yr ACS census
sf = shapefile.Reader("/Users/23amrutad/Projects/LeadProj/ProjectData/cb_2019_33_tract_500k/cb_2019_33_tract_500k.shp")
shapes = sf.shapes()
records = sf.records()

# name_map should have the correct tract ID for each school, based on the data from shapes
name_map_new = {}
tracts = []
for k, v in name_map.items():
    if v['lat'] != '' and v['long'] != '':
        v_new = v
        tract_num = findTract(v['lat'], v['long'], shapes, records)
        v_new['tract'] = tract_num
        name_map_new[k] = v_new
        tracts.append(v_new['tract'])
name_map = name_map_new

In [5]:
# part 1 of data; includes the school name, address, tract number, latitude, longitude (doesn't include census data)
# need to make further modifications, can restore initial data from here

initial_path = "/Users/23amrutad/Projects/LeadProj/ProjectData/initial_data.pkl"
initial = open(initial_path, 'wb')
pickle.dump(name_map, initial)
initial.close()

In [6]:
# these files are downloaded from the Census Data API
# the path_endings list contains the names of the tables downloaded
# these files are read in as dataframes and merged together into one super dataframe with all the 
# necessary census data

path_first = "/Users/23amrutad/Projects/LeadProj/CensusFiles/"
path_endings = ['B01001', 'C17002', 'B14006', 'B17001', 'B06011', 'B11001', 'B19083', 'B25096', 'B15003', 
               'B25075', 'B14007', 'B19057', 'B25003', 'B25050', 'B25034', 'B09019']
df = pd.read_csv("/Users/23amrutad/Projects/LeadProj/CensusFiles/B02001/B02001.csv")
for path in path_endings:
    complete_path = path_first + path + "/" + path + ".csv"
    df1 = pd.read_csv(complete_path)
    df = df.merge(df1)

In [7]:

'''
this function takes in a dictionary and a dataframe,
and returns the row in the dataframe the corresponds to an entry in the dictinoary

specificially, the dataframe is census data and the dictionary contains the details of the school,
the tract number from the dicitionary is used to determine whether the school is actually in the dataframe.
if it is, all the info is returned
'''

def returnCensusInfo(val : dict, census_df):
    target_tract = val['tract']
    
    for r in census_df.iterrows():
        # extract tract id from the row
        t = r[1]['GEO_ID']
        if t == 'id':
            continue
        #print("%s ==? %s"%(t[14:], target_tract))
        if len(t) > 14 and t[14:] == target_tract:
            # only when the lower 6 characters of the tract match, then copy
            # the value of each attribute
            features = (r[1].keys().tolist())
            for x in features:
                val[x] = r[1].get(x)
            break
    return val

# loading the previously saved data to combine it with the census data
a = open(initial_path, 'rb')
name_map = pickle.load(a)

name_map_copy = {}
for k, v in name_map.items():
    newinfo = returnCensusInfo(v, df)
    if newinfo is not None:
        name_map_copy[k] = newinfo
        
name_map_df = pd.DataFrame.from_dict(name_map)
name_map = name_map_copy

In [8]:
# converting every value in name_map to float

for r in name_map.keys():
    v = name_map[r]
    for k in v.keys():
        try:
            v[k] = float(v[k])
        except:
            v[k] = 0.0
    name_map[r] = v
name_map_df = pd.DataFrame.from_dict(name_map)

In [10]:
# saving the updated, fully populated, and float-only name_map to a pickle file.
# also saving a version as a spreadsheet, for visual inspection.

dbfile = open("/Users/23amrutad/Projects/LeadProj/ProjectData/final_data.pkl", 'wb')
pickle.dump(name_map, dbfile)
dbfile.close()
name_map_df.to_excel("/Users/23amrutad/Projects/LeadProj/ProjectData/final_data.xlsx")